In [1]:
# imports
import pandas as pd
import json
import os
from IPython.core.debugger import set_trace
# change directory
# os.chdir(os.path.dirname(os.getcwd()))

In [2]:
os.chdir('c:\\Users\\sonny\\Documents\\Data Science\\TechLabs Project\\src')

In [3]:
# settings for Lottie (Animations)

data_points_url = 'https://assets5.lottiefiles.com/packages/lf20_6dboqita.json'
wondering_url = 'https://assets4.lottiefiles.com/packages/lf20_pcoatxlk.json'
typing_url = 'https://assets1.lottiefiles.com/temporary_files/r5WAZZ.json'
reportlist_url='https://assets9.lottiefiles.com/packages/lf20_dq6whaxi.json'
options = dict(loop=True, autoplay=True, rendererSettings=dict(preserveAspectRatio='xMidYMid slice'))

In [4]:
# set application title
application_name = 'BraSolar' 
report_status = ('Status updates will be displayed after starting the analysis.\nAnalysis at 18% - weather data loaded.\nAnalysis at 23%')

In [5]:
# import libraries & components

# import custom components
# in the future the source will need to be changed (add d07_visualisation in front of 'components')
from components import start_button, no_fig, your_fig

# 
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc 
from dash_extensions import Lottie
import pandas as pd
import os
from dash import no_update
from dash.exceptions import PreventUpdate
from time import sleep, perf_counter
import geopandas as gpd


In [6]:
# custom cardbody for status report
status_card = dbc.CardBody([
    html.H4('Status report'),
    html.P('Status updates will be displayed after starting the analysis.'),
    html.P('Analysis at 18% - weather data loaded.')

],id='status_card',style={'font_family':'Courier'})

In [7]:
# custom html.P wrapper to include status reports
status_p = html.P(['Status updates will be displayed after starting the analysis.'])

In [8]:
status_p.children = status_p.children+[html.Br(),'lololo']

In [9]:
status_p

P(['Status updates will be displayed after starting the analysis.', Br(None), 'lololo'])

In [10]:
# Bootstrap themes by Ann: https://hellodash.pythonanywhere.com/theme_explorer
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.CYBORG])
app.title = application_name

app.layout = dbc.Container([
    # row one
    dbc.Row([
        # row one col one
        dbc.Col([
            # logo
            dbc.Card([
                dbc.CardImg(src='../references/solarlogo.png')
            ],className='mb-2'),
            # name and link to website
            dbc.Card([
                dbc.CardBody([
                    dbc.CardLink('Sönke Maibach',target='_blank',  # _blank opens a new tab on click
                    href='https://mabikono.de')
                ])
            ],style={'height':'16vh'}),
        ], width=2),
        # row one col two
        dbc.Col([
            # title and short info
            dbc.Card([
                dbc.CardBody([
                    html.H4('{}'.format(application_name)),
                    html.P('Find out if your chosen locations are suitable for solar power.')
                ])
            ], color = 'info',style={'height':'18vh'}),
        ], width=6),
        # row one col three
        # Dark/Light Mode and Language Dropdown
        dbc.Col([
            dbc.Card([
                dbc.Button('Turn on light',color='primary', id='theme_switch')
            ])
        ],width=2)
    ], className='m-2 mt-4 h-25'),
    # row two
    dbc.Row([
        # row two col one
        # description
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(Lottie(options=options,url=wondering_url, width='40%', height='40%')),
                dbc.CardBody([
                    html.H4('What is going on here?',className='card-title'),
                    html.P(
                        'Computing energy cost savings and ROI by installing '
                        'a solar system. Uses weather and topographical data '
                        'combined with personal consumption profile to find '
                        'location, tilt and alignment with maximum energy output, '
                        'energy cost savings or ROI.'
                    )
                ])
            ]),
        ], width=4),
        # row two col two
        # inputs and start button
        dbc.Col([
            dbc.Card([
                # card header for the inputs
                dbc.CardHeader(
                    Lottie(options=options,url=typing_url, height='25%', width='25%')
                ),
                dbc.CardBody([
                    html.H4('Tell me a bit about yourself'),
                    # input field for location
                    html.I("Choose a city, region or country."),
                    html.Br(),
                    dcc.Input(id="location_input", type="search",
                    value="Sachsen",
                    style={'marginRight':'20px'}),
                    html.Br(), html.Br(),

                    
                    
                    # radioitems resolution
                    html.I("Choose the resolution. A higher resolution means more data points (DP) will be analyzed."),
                    html.Br(),
                    dcc.RadioItems(id='resolution_radio',
                        options=[
                            {'label':'Fastest (~100 DP)','value':200},
                            {'label':'Balanced (~400 DP)','value':800},
                            {'label':'Very detailed (~1000 DP)','value':2000},
                        ],
                        value=200,
                        labelStyle={'display': 'block'}
                    ),

                    # Household Size input
                    html.I('How many people live in your household? Children count as 0.5.'),
                    html.Br(),
                    dcc.Input(id="household_input", type="number",
                    value=3,
                    style={'marginRight':'20px'}),
                    html.Br(), html.Br(),

                    # start button
                    start_button,

                    # choose output to plot
                    html.I('Choose what you want to see.'), html.Br(),
                    dcc.RadioItems(
                        options=[
                            {'label':'Maximum electricity output','value':'max_out'},
                            {'label':'Maximum energy cost savings','value':'max_savings'},
                            {'label':'Maximum ROI','value':'max_roi'},
                        ],
                        value='max_roi',
                        labelStyle={'display':'block'},
                        id='to_plot_radio'
                    )

                ])
            ]),
        ], width=4),
        # row two col three
        # status updates
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(Lottie(url=reportlist_url,options=options,height='20%',width='20%')),
                dbc.CardBody([
                    html.H4('Status report'),
                    html.P('Status updates will be displayed after starting the analysis.',id='status_p'),
                    html.Span(id='status_span',children='If you see this message, function display_status is probably broken')
                ],id='status_card')
            ]),
        ], width=4),
    ],className='m-2 h-15'),
    # row three 
    dbc.Row([
        # row three col one
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(id='map_graph',figure={}),
                ])
            ]),
        ], width=6,style={}),
        # row three col two
        # additional graph next to map
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(id='histogram_graph',figure={}),
                ])
            ]),
        ], width=6),
    ],className='m-2',style={}),

    dbc.Row([
        # row four col one
        # graph to the left
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(id='ll_graph',figure={}),
                ])
            ]),
        ], width=3),
        # row three col two
        # centered graph
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(id='lm_graph',figure={}),
                ])
            ]),
        ], width=5),
        # row three col three
        # graph to the right
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(id='lr_graph',figure={}),
                ])
            ]),
        ], width=4),
        # hidden data storage
        dcc.Store(id='status_store',data=['pre']),
        # variables to check if single steps are done
        dcc.Store(id='location_reference_store'),
        dcc.Store(id='weather_raw_store'),

        # check if weather and loc reference are available, create and save them if needed
        dcc.Store(id='weather_loaded_store'),
        # get topo data, compute slope
        dcc.Store(id='topo_loaded_store'),
        # get tilt and alignment suggestions
        dcc.Store(id='tiltalign_store'),
        # compute sun geometry, including angle of incidence
        dcc.Store(id='sungeo_store'),
        # compute loss by soiling, snow, temp and all combined
        dcc.Store(id='losses_store'),
        # get setup for maximum output and optimal personal use
        dcc.Store(id='setup_store'),
        # get financial data for all setups
        dcc.Store(id='financial_store',data=['finstart']),
        # prepare visualisation, polygons will be stored here
        dcc.Store(id='visual_store'),
        dcc.Store(id='analysis_done_store',data=[False]),

        dcc.Store(id='started_store',data=[False]),
        dcc.Store(id='step_done_store'),
        dcc.Store(id='weather_df_store'),
        dcc.Store(id='max_out_store'),
        dcc.Store(id='smallest_area_store'),
        dcc.Store(id='user_input_store'),
        dcc.Store(id='folder_path_store')
    ],className='m-2 mb-4',style={}),
    

], fluid=True, style={'height':'100vh'})

Callbacks

In [11]:
# reset outputs when changing user input
@app.callback(
    Output('start_button','n_clicks'),
    Input('location_input','value'),
    Input('resolution_radio','value'),
    Input('household_input','value')
)
def reset_analysis(loc,res,hh):
    return 0

In [12]:
# # start button click should lead to start of the analysis and show the reference location
# @app.callback(
#     Output('status_store','data'),
#     Output('histogram_graph','figure'),
#     Input("start_button", "n_clicks"),
#     State('location_input','value'),
#     State('resolution_radio','value'),
#     State('household_input','value')
# )
# def letsgo(clicks,loc,res,hh):
#     if clicks == 0:
#         return ['pre new func'], no_fig
#     else:
#         return ['started {clicks}'], no_fig

In [13]:
# # prepare visulaisation (last step of the analysis)
# @app.callback(
#     Output('analysis_done_store','data'),
#     Output('status_store','data'),
#     # trigger when the second to last step is done
#     Input('financial_store','data'),
#     State('location_input','value'),
#     State('resolution_radio','value'),
#     State('max_out_df_store','data'),
#     State('status_store','data')
# )
# def prepare_polygons(fins,loc,res,df,status):
#     print('final step triggered, this should happen only after computing finances')
#     starttime = perf_counter()
#     path = '../references/polygons_'+loc+'_'+str(res)+'.json'
#     # if the relevant polygons are not available, take them from the df and save them
#     if not os.path.exists(path):
#         polygon_json = gpd.GeoSeries(df.geometry).__geo_interface__
#         with open(path,'w') as f:
#             json.dump(polygon_json,f)
#     # set the final status variable to True
#     steptime = starttime -perf_counter()
#     return [True], ['Analysis at 100% - Visualisation prepared in {steptime:.2f} seconds']

# # after the final step of the analysis, show the results
# @app.callback(
#     # map output
#     Output('map_graph','figure'),
#     # right output
#     Output('histogram_graph','figure'),
#     # last row outputs
#     Output('ll_graph','figure'),
#     Output('lm_graph','figure'),
#     Output('lr_graph','figure'),
#     # this step should be triggered after the final step or when the relevant output is changed
#     Input('analysis_done_store','data'),
#     Input('to_plot_radio','value'),
#     State('location_input','value'),
#     State('resolution_radio','value'),
#     State('household_radio','value')

# )
# def show_results(done,chosen_output,loc,res,hh):
#     if not done[0]:
#         return no_fig,no_fig,no_fig,no_fig,no_fig
#     else:
#         print('starting to plot')
#         # load files
#         df_path = '../data/03_processed/'+chosen_output+'_'+loc+'_'+str(res)+'_hh'+str(round(hh*10))+'.pqt'
#         df = pd.read_parquet(df_path)
#         polygons_path = '../references/polygons_'+loc+'_'+str(res)+'.json'
#         with open (polygons_path) as f:
#             polygons = json.load(f)
#             f.close()




#         # create map
#         map_fig = px.choropleth_mapbox(df, geojson=polygons, locations='index', color='loss_temp',
#                                     color_continuous_scale="Viridis",
#                                     #    range_color=(0.1, .11),
                                    
#                                     mapbox_style="carto-darkmatter",
#                                     zoom=6, center = {
#                                         "lat": polygons['features'][int(100/2)]['bbox'][1],
#                                         "lon": polygons['features'][int(100/2)]['bbox'][0]
#                                     },
#                                     opacity=0.8,
#                                     labels={'loss_temp':'loss by temp',
#                                                 'loss_soiling':'loss by dirt'},
#                                     hover_data={'loss_soiling':':.2f'
#                                                 #,'loss_by_temp':':.2f'
#                                                 },
#                                     height=600
#                                     )
#         map_fig.update_layout(
#             margin={"r":0,"t":0,"l":0,"b":0}
#         )
#         print('can you see it?')
#         return map_fig,no_fig,no_fig,no_fig,no_fig

In [14]:
# start button
# after clicking the start button, the callback will check the status of the analysis
# the status will be used to continue with the next step in the analysis
@app.callback(
    Output('status_store','data'),
    Input("start_button", "n_clicks"),
    State('location_input','value'),
    State('resolution_radio','value'),
    State('household_input','value'),
    State('status_store','data'),
    State('financial_store','data')
)
def check_status(clicks,loc,res,hh,status,finstat):
    # set paths for comparison
    smallest_size_df_path = '../data/03_processed/smallest_area_'+loc+'_'+str(res)+'_hh'+str(round(hh*10))+'/'
    user_input = [{'loc':loc, 'res':res, 'hh':hh}]
    if clicks == 0:
        # return pre if analysis has not been started
        return ['pre']
    # check if different files exist and set the status accordingly
    # first check the last file to be created, if it's there the analysis has been run before
    elif os.path.exists('../data/07_visualisation/'+loc+'_'+str(res)+'_hh'+str(round(hh*10))+'polygons.json'):
        return ['done']
    # check the first file to be created, if it's missing the analysis needs to be run fully
    elif os.path.exists('../references/'+loc+'_loc_ref.json'):
        return ['loc_ref']
    
    # pls include more options for files here

    elif clicks == 4:
        no_update
    else:
        return ['started'+status[0]+status[0]+str(finstat)]

In [15]:
# simulation
@app.callback(
    Output('to_plot_radio','value'),
    Input('status_store','modified_timestamp'),
    State('status_store','data')
)
def testfunc(ts,status):
    if status is None:
        print('nah')
    elif status[0] == 'started':
        print('yo')
        return 'max_savings'
    else:
        print('ye')
        return 'max_out'

In [16]:
# update the visible status report based on the status_store
@app.callback(
    Output('status_span','children'),
    Input('status_store','data')
)
def display_status(status):
    return status[0]

In [17]:
# start button
# hashtagged to test final step func 
@app.callback(
    # test output
    Output("example-output", "children"),
    # output map
    Output('map_graph','figure'),

    # for testing purposes dont update hist but another variable
    # right output
    Output('financial_store','data'),
    # Output('histogram_graph','figure'),


    # last row outputs
    Output('ll_graph','figure'),
    Output('lm_graph','figure'),
    Output('lr_graph','figure'),
    Input("start_button", "n_clicks"),
    State('location_input','value')
)
def on_button_click(n,loc):
    if n ==0:
        return ["Not clicked."], no_fig, no_fig, no_fig, no_fig, no_fig
    else:
        # first status update
        status_p.children = ['Analysis started']

        # this is a fake path for testing purposes
        max_path ='../data/03_processed/max_out_'+loc.lower()+'.pqt'
        if os.path.exists(max_path):
            test_df = pd.read_parquet(max_path).reset_index()
        geo_path = '../references/polygons_'+loc.lower()+'.json'
        with open(geo_path) as gfile:
            jsonObject = json.load(gfile)
            gfile.close()

        # create map
        map_fig = px.choropleth_mapbox(test_df, geojson=jsonObject, locations='index', color='loss_temp',
                                color_continuous_scale="Viridis",
                                #    range_color=(0.1, .11),
                                
                                mapbox_style="carto-darkmatter",
                                zoom=6, center = {
                                    "lat": jsonObject['features'][int(100/2)]['bbox'][1],
                                    "lon": jsonObject['features'][int(100/2)]['bbox'][0]
                                },
                                opacity=0.4,
                                labels={'loss_temp':'loss by temp',
                                            'loss_soiling':'loss by dirt'},
                                hover_data={'loss_soiling':':.2f'
                                            #,'loss_by_temp':':.2f'
                                            },
                                height=600
                                )
        map_fig.update_layout(
            margin={"r":0,"t":0,"l":0,"b":0}
        )


        # for testing purposes dont update hist, use fake value
        hist_fig = ['finupdate']
        # create histogram next to map
        # hist_fig = px.histogram(test_df,x='savings_cost_ratio')


        # create scatter plot for ll_fig
        ll_fig = px.scatter(test_df.iloc[:10*n+40],x='loss_soiling',y='loss_temp')
        counter = [f"Clicked {n} times." + str(test_df.iloc[5,n])]
        
        return counter, map_fig, hist_fig, ll_fig, your_fig, your_fig

In [18]:
# test if output of map func can trigger another output
@app.callback(
    Output('histogram_graph','figure'),
    Input('financial_store','data')
)
def test_var_trigger(fins):
    print('trigger successful')
    return no_fig

In [19]:
# run the server
# server can also be accessed via browser (http://127.0.0.1:8050/)
app.run_server(mode='inline',debug=True)

OSError: Address 'http://127.0.0.1:8050' already in use.
    Try passing a different port to run_server.